In [11]:
# ============================================================
# EEG Foundation Challenge 2025 - Challenge 1
# SuS pretraining → CCD RT regression (100 Hz preprocessed version)
# ------------------------------------------------------------
# - Reads *_eeg_pp.set EEGs (100 Hz) from per-subject folders (run optional)
# - Matches CCD events from BIDS (ds*/sub-*/eeg/*_events.tsv)
# - Caches normalized EEG (.npy)
# - Suppresses MNE/User/Future/Runtime warnings
# - Safe EEGConformer wrapper (version differences)
# ============================================================

import os, random, numpy as np, pandas as pd, warnings, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from glob import glob
import torch.nn.functional as F

# ---- suppress warnings/logs ----
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import mne
mne.set_log_level("ERROR")

# ============================================================
# 0. Config (경로만 맞춰주세요)
# ============================================================
BIDS_ROOT         = "/data5/open_data/HBN/EEG_BIDS/"
PREPROCESSED_ROOT = "/data5/open_data/HBN/Preprocessed_EEG/0922try_bySubject/"
CACHE_DIR         = "/data5/open_data/HBN/cache_eeg_100hz_noref"
os.makedirs(CACHE_DIR, exist_ok=True)

TARGET_SFREQ = 100
WIN_S_SUS, WIN_S_CCD = 2.0, 2.0            # 윈도 길이(초)
STRIDE_S_SUS = 1.0                         # SuS pretrain용 윈도 stride(초)
BATCH_SIZE, NUM_WORKERS = 64, 2
EPOCHS_SUS, EPOCHS_CCD, LR_SUS, LR_CCD = 5, 10, 1e-3, 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ============================================================
# 1) File collectors (run 유무 포함)
# ============================================================
def collect_preprocessed_files(root_path, task_name=None):
    """
    /.../bySubject/sub-XXXX/** 에서 *_eeg_pp.set 수집 (run 유무 무관)
    task_name이 주어지면 해당 문자열 포함 파일만.
    """
    results = []
    for dirpath, _, filenames in os.walk(root_path):
        # subject 폴더만 탐색
        if "sub-" not in dirpath: 
            continue
        for fn in filenames:
            low = fn.lower()
            if not low.endswith("_eeg_pp.set"):
                continue
            if task_name and (task_name.lower() not in low):
                continue
            results.append(os.path.join(dirpath, fn))
    results = sorted(results)
    print(f"[INFO] Found {len(results)} preprocessed EEG files ({'task: '+task_name if task_name else 'all'})")
    return [(f, "") for f in results]

def collect_ccd_event_files(bids_root):
    """BIDS 폴더에서 CCD 이벤트 파일 몽땅 수집."""
    ev_files = glob(os.path.join(
        bids_root, "ds*/sub-*", "eeg", "sub-*_task-contrastChangeDetection*_events.tsv"
    ))
    print(f"✅ Found {len(ev_files)} CCD event files.")
    return ev_files

def match_eeg_to_event(preproc_files, bids_root):
    """
    preprocessed CCD EEG ↔ BIDS 이벤트 매칭.
    규칙: *_eeg_pp.set → *_events.tsv (run 유무 모두 대응)
    """
    ev_files = collect_ccd_event_files(bids_root)
    ev_dict = {os.path.basename(ef).replace("_events.tsv", ""): ef for ef in ev_files}

    pairs = []
    for eeg_path, _ in preproc_files:
        base = os.path.basename(eeg_path)
        # pp 접미어 제거하여 키 생성 (확장자 제거)
        key = base.replace("_eeg_pp.set", "").replace(".set", "")
        if key in ev_dict:
            pairs.append((eeg_path, ev_dict[key]))
        else:
            # run 없는 변형도 탐색
            key_no_run = key.replace("_run-1", "").replace("_run-2", "")
            matched = None
            for k, v in ev_dict.items():
                k_norm = k.replace("_run-1", "").replace("_run-2", "")
                if k_norm == key_no_run:
                    matched = v; break
            if matched:
                pairs.append((eeg_path, matched))
    print(f"🔗 Matched {len(pairs)} EEG ↔ event pairs.")
    return pairs

# ============================================================
# 2) Cached EEG loader (z-score only, no align/resample)
# ============================================================
def read_raw(eeg_path):
    return mne.io.read_raw_eeglab(eeg_path, preload=True, verbose=False)

def cached_load_eeg(eeg_path):
    """
    파일 단위 캐시: <basename>_cached.npy
    [수정] .npy 파일이 존재하면 로드하고, 없으면 None을 반환합니다.
    """
    fname = os.path.basename(eeg_path).replace(".set", "_cached.npy")
    cache_path = os.path.join(CACHE_DIR, fname)
    
    if os.path.exists(cache_path):
        return np.load(cache_path)
    else:
        # .set 파일에서 새로 생성하는 로직을 제거함
        return None

def make_window(x_ct, center_s, sfreq=TARGET_SFREQ, win_sec=2.0):
    """center 이전 win_sec 구간을 잘라 [C,Tw] 반환(부족하면 좌측 패딩)."""
    t1 = int(center_s * sfreq)
    Tw = int(win_sec * sfreq)
    t0 = max(0, t1 - Tw)
    seg = x_ct[:, t0:t1]
    need = Tw - seg.shape[1]
    if need > 0:
        seg = np.pad(seg, ((0, 0), (need, 0)), mode="constant")
    return seg.astype(np.float32)

# ============================================================
# 3) CCD trial parser (correct only: feedback==smiley)
# ============================================================
def extract_ccd_trials(df):
    if df.empty or "onset" not in df.columns or "value" not in df.columns:
        return []
    trials = []
    on  = df["onset"].astype(float).values
    val = df["value"].astype(str).values
    fb  = df["feedback"].astype(str).values if "feedback" in df.columns else ["n/a"] * len(df)
    starts  = [i for i,v in enumerate(val) if "contrastTrial_start" in v]
    presses = [i for i,v in enumerate(val) if "buttonPress" in v]
    for ti in starts:
        t0 = on[ti]
        later = [pi for pi in presses if on[pi] > t0]
        if not later: 
            continue
        pi = later[0]
        rt = (on[pi]-t0) * 1000.0
        if 100 <= rt <= 3000 and "smiley" in fb[pi].lower():
            trials.append((t0, rt))
    return trials

# ============================================================
# 4) Datasets
# ============================================================
class SusPretrainDataset(Dataset):
    """
    이벤트 없이 SuS 파일에서 윈도우를 규칙적/랜덤 추출해 두 view로 반환.
    """
    def __init__(self, eeg_files, win_s=WIN_S_SUS, stride_s=STRIDE_S_SUS, random_start=True):
        self.items = []  # (eeg_path, center_s)
        self.win_s = win_s; self.stride_s = stride_s; self.random_start = random_start
        
        print("Building SuS dataset from .npy cache...")
        missing_count = 0
        for p,_ in tqdm(eeg_files, desc="Scanning SuS cache"):
            X = cached_load_eeg(p) # .npy 로드 시도
            
            if X is None:
                missing_count += 1
                continue  # .npy가 없으면 이 파일 스킵
                        
            T = X.shape[1]
            Tw = int(win_s * TARGET_SFREQ)
            stride = int(stride_s * TARGET_SFREQ)
            centers = []
            if T > Tw:
                for t1 in range(Tw, T, stride):
                    centers.append(t1 / TARGET_SFREQ)
            if len(centers) == 0 and T >= Tw:
                centers = [Tw / TARGET_SFREQ]
            for c in centers:
                self.items.append((p, c))
        
        if missing_count > 0:
            print(f"⚠️  Skipped {missing_count} SuS files (missing .npy cache).")
        print(f"✅ SuS Dataset: {len(self.items)} valid cached windows.")
        random.shuffle(self.items)

    def __len__(self): return len(self.items)

    @staticmethod
    def _augment(x):
        # 간단한 증강: 가우시안 노이즈 + 타임마스크 + 채널 드롭
        x = x + 0.01 * np.random.randn(*x.shape).astype(np.float32)
        if np.random.rand() < 0.5:
            L = max(1, int(x.shape[1]*0.1))
            s = np.random.randint(0, x.shape[1]-L+1)
            x[:, s:s+L] = 0.0
        if np.random.rand() < 0.5:
            drop = max(1, int(x.shape[0]*0.05))
            idx = np.random.choice(x.shape[0], drop, replace=False)
            x[idx] = 0.0
        return x

    def __getitem__(self, idx):
        p, c = self.items[idx]
        X = cached_load_eeg(p) # .npy 로드 (init에서 확인했으므로 항상 존재)
        
        if X is None:
            # (안전 장치) 캐시가 그새 삭제된 경우
            Tw = int(self.win_s * TARGET_SFREQ)
            # 채널 수를 128로 가정 (C, T)
            v1 = np.zeros((128, Tw), dtype=np.float32) 
            v2 = np.zeros((128, Tw), dtype=np.float32)
            return torch.from_numpy(v1), torch.from_numpy(v2), torch.zeros(1)

        seg = make_window(X, c, win_sec=self.win_s)
        v1 = self._augment(seg.copy())
        v2 = self._augment(seg.copy())
        return torch.from_numpy(v1), torch.from_numpy(v2), torch.zeros(1)

class CcdRtDataset(Dataset):
    def __init__(self, eeg_event_pairs, win_s=WIN_S_CCD):
        all_samples = []  # (eeg_path, onset_s, rt_ms)
        
        print("Building CCD dataset from event files...")
        for eeg_path, ev_path in tqdm(eeg_event_pairs, desc="Parsing CCD events"):
            if not os.path.exists(ev_path): 
                continue
            df = pd.read_csv(ev_path, sep="\t")
            for o, rt in extract_ccd_trials(df):
                all_samples.append((eeg_path, o, rt))

        # --- [ ⭐️ .npy 캐시 검증 단계 추가 ⭐️ ] ---
        print(f"Found {len(all_samples)} total trials. Verifying .npy cache...")
        self.samples = [] # .npy가 존재하는 샘플만 추가
        missing_files = set()
        
        for (eeg_path, o, rt) in tqdm(all_samples, desc="Verifying CCD cache"):
            # .npy 파일 존재 여부만 확인 (로드X)
            fname = os.path.basename(eeg_path).replace(".set", "_cached.npy")
            cache_path = os.path.join(CACHE_DIR, fname)
            
            if os.path.exists(cache_path):
                self.samples.append((eeg_path, o, rt))
            else:
                missing_files.add(eeg_path)
        
        if missing_files:
            print(f"⚠️  Skipped {len(all_samples) - len(self.samples)} CCD trials (missing .npy from {len(missing_files)} files).")
        # --- [ ⭐️ 검증 끝 ⭐️ ] ---
        
        if not self.samples:
            print("❌ ERROR: No valid cached CCD samples found!")
            self.rt_mean = 0.0
            self.rt_std = 1.0
        else:
            all_rts = np.array([s[2] for s in self.samples]).astype(np.float32)
            self.rt_mean = all_rts.mean()
            self.rt_std = all_rts.std() + 1e-6
        
        print(f"✅ CCD Dataset: {len(self.samples)} valid cached trials. RT(ms) Mean={self.rt_mean:.2f}, Std={self.rt_std:.2f}")
        self.win_s = win_s

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        p, o, rt = self.samples[idx]
        X = cached_load_eeg(p) # .npy 로드 (init에서 확인했으므로 항상 존재)
        
        if X is None:
            # (안전 장치)
            Tw = int(self.win_s * TARGET_SFREQ)
            seg = np.zeros((128, Tw), dtype=np.float32)
            rt_normalized = 0.0
        else:
            seg = make_window(X, o, win_sec=self.win_s)
            rt_normalized = (rt - self.rt_mean) / self.rt_std
        
        return torch.from_numpy(seg), torch.tensor([rt_normalized], dtype=torch.float32)

# ============================================================
# 5) EEGConformer encoder (safe wrapper)
# ============================================================
from braindecode.models import EEGConformer

class SafeEEGConformerEncoder(nn.Module):
    """
    EEGConformer 버전별 생성자 차이 자동 대응 + 출력 flatten.
    """
    def __init__(self, n_chans, sfreq, input_window_samples):
        super().__init__()
        last_err = None
        trials = [
            dict(n_chans=n_chans, n_outputs=1, n_times=input_window_samples,              sfreq=sfreq, return_features=True),
            dict(n_chans=n_chans, n_outputs=1, input_window_samples=input_window_samples, sfreq=sfreq, return_features=True),
            dict(n_chans=n_chans, n_outputs=1, n_times=input_window_samples,              sfreq=sfreq),
        ]
        for kw in trials:
            try:
                self.backbone = EEGConformer(**kw)
                break
            except TypeError as e:
                last_err = e
        if not hasattr(self, "backbone"):
            raise TypeError(f"EEGConformer init failed. Last error: {last_err}")

    def forward(self, x):
        z = self.backbone(x)
        if isinstance(z, tuple): z = z[0]
        return torch.flatten(z, 1)

class ContrastiveHead(nn.Module):
    def __init__(self, in_dim, proj_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, in_dim//2),
            nn.ReLU(),
            nn.Linear(in_dim//2, proj_dim)
        )
    def forward(self, x):
        if x.ndim > 2: x = torch.flatten(x, 1)
        return self.net(x)


# ============================================================
# 6) Heads & Losses
# ============================================================
class RtHead(nn.Module):
    def __init__(self, feat_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(feat_dim, max(64, feat_dim//2)),
            nn.ReLU(),
            nn.Linear(max(64, feat_dim//2), 1)
        )
    def forward(self, z):
        if z.ndim > 2: z = torch.flatten(z, 1)
        return self.mlp(z)

def nt_xent_loss(z1, z2, temperature: float = 0.5):
    z1 = F.normalize(z1, dim=1)
    z2 = F.normalize(z2, dim=1)

    logits12 = torch.matmul(z1, z2.T) / temperature   # (N, N)
    logits21 = torch.matmul(z2, z1.T) / temperature   # (N, N)

    # 안정화
    logits12 = logits12 - logits12.max(dim=1, keepdim=True).values
    logits21 = logits21 - logits21.max(dim=1, keepdim=True).values

    labels = torch.arange(z1.size(0), device=z1.device)  # diag가 positive
    loss = (F.cross_entropy(logits12, labels) + F.cross_entropy(logits21, labels)) / 2
    return loss


# ============================================================
# 7) Train loops
# ============================================================
def train_pretrain_sus(dl, encoder, epochs=EPOCHS_SUS, lr=LR_SUS):
    # feature projection 추가
    with torch.no_grad():
        dummy, _, _ = next(iter(dl))
        feat_dim = encoder(dummy[:1].float().to(DEVICE)).shape[1]
    proj_head = nn.Sequential(
        nn.Linear(feat_dim, feat_dim // 2),
        nn.ReLU(),
        nn.Linear(feat_dim // 2, 128)
    ).to(DEVICE)

    opt = torch.optim.Adam(list(encoder.parameters()) + list(proj_head.parameters()), lr=lr)
    for ep in range(epochs):
        encoder.train(); proj_head.train(); losses=[]
        for x1, x2, _ in tqdm(dl, desc=f"[SuS pretrain] {ep+1}/{epochs}"):
            x1, x2 = x1.float().to(DEVICE), x2.float().to(DEVICE)
            z1, z2 = encoder(x1), encoder(x2)
            p1, p2 = proj_head(z1), proj_head(z2)
            loss = nt_xent_loss(p1, p2, temperature=2.0)
            opt.zero_grad(); loss.backward(); opt.step()
            losses.append(loss.item())
        print(f"Epoch {ep+1}: contrastive loss={np.mean(losses):.4f}")
    return encoder


def train_ccd_rt(dl_tr, encoder, rt_head, epochs=EPOCHS_CCD, lr=LR_CCD):
    opt = torch.optim.Adam(list(encoder.parameters()) + list(rt_head.parameters()), lr=lr)
    for ep in range(epochs):
        encoder.train(); rt_head.train(); losses=[]
        for x, y in tqdm(dl_tr, desc=f"[CCD train] {ep+1}/{epochs}"):
            x, y = x.float().to(DEVICE), y.to(DEVICE)
            yhat = rt_head(encoder(x))
            loss = nn.functional.l1_loss(yhat, y)
            opt.zero_grad(); loss.backward(); opt.step()
            losses.append(loss.item())
        print(f"Epoch {ep+1}: MAE={np.mean(losses):.3f} ms")

# ============================================================
# 8) Main
# ============================================================
def main():
    # --- 1) SuS pretraining ---
    sus_files = collect_preprocessed_files(PREPROCESSED_ROOT, task_name="surroundSupp")
    ds_sus = SusPretrainDataset(sus_files, win_s=WIN_S_SUS, stride_s=STRIDE_S_SUS)
    dl_sus = DataLoader(ds_sus, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

    # encoder 초기화 (SuS 배치로 C,T 추정)
    x_demo, _, _ = next(iter(dl_sus))
    _, C, T = x_demo.shape
    encoder = SafeEEGConformerEncoder(C, TARGET_SFREQ, T).to(DEVICE)

    # pretrain
    encoder = train_pretrain_sus(dl_sus, encoder, epochs=EPOCHS_SUS, lr=LR_SUS)

    # --- 2) CCD fine-tuning (RT) ---
    ccd_eeg_files = collect_preprocessed_files(PREPROCESSED_ROOT, task_name="contrastChangeDetection")
    matched_pairs = match_eeg_to_event(ccd_eeg_files, BIDS_ROOT)
    ds_ccd = CcdRtDataset(matched_pairs, win_s=WIN_S_CCD)
    dl_ccd = DataLoader(ds_ccd, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

    # head 만들고 학습
    with torch.no_grad():
        feat_dim = encoder(x_demo[:1].float().to(DEVICE)).shape[1]
    print(f"[INFO] Encoder feature dim: {feat_dim}")
    rt_head = RtHead(feat_dim).to(DEVICE)

    train_ccd_rt(dl_ccd, encoder, rt_head, epochs=EPOCHS_CCD, lr=LR_CCD)

    # 학습 종료 후 encoder + rt_head 저장
    torch.save({
        "encoder": encoder.state_dict(),
        "rt_head": rt_head.state_dict()
    }, "weights_ch1.pth")

    print("✅ Saved Challenge 1 weights to weights_ch1.pth")


if __name__ == "__main__":
    main()


[INFO] Found 4149 preprocessed EEG files (task: surroundSupp)
Building SuS dataset from .npy cache...


Scanning SuS cache: 100%|██████████| 4149/4149 [03:23<00:00, 20.41it/s]


✅ SuS Dataset: 1725089 valid cached windows.


[SuS pretrain] 1/5:  16%|█▌        | 4306/26955 [1:46:16<7:51:25,  1.25s/it] Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd2d9897eb0>
Traceback (most recent call last):
  File "/home/RA/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/RA/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd2d9897eb0>
Traceback (most recent call last):
  File "/home/RA/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/RA/.local/lib/python3.10/site-packages/torch/

Epoch 1: contrastive loss=3.7075


[SuS pretrain] 2/5:  16%|█▌        | 4280/26955 [1:32:07<9:23:33,  1.49s/it] Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd2d9897eb0>
Traceback (most recent call last):
  File "/home/RA/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/RA/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd2d9897eb0>
Traceback (most recent call last):
  File "/home/RA/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/RA/.local/lib/python3.10/site-packages/torch/

Epoch 2: contrastive loss=3.6994


[SuS pretrain] 3/5:  16%|█▌        | 4376/26955 [1:08:04<8:29:38,  1.35s/it] Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd2d9897eb0>
Traceback (most recent call last):
  File "/home/RA/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/RA/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
[SuS pretrain] 3/5:  16%|█▌        | 4377/26955 [1:08:07<10:22:24,  1.65s/it]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd2d9897eb0>
Traceback (most recent call last):
  File "/home/RA/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._s

In [10]:
# ============================================================
# Challenge 1 - CCD downstream using pretrained CBraMod encoder
# ============================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np

# ------------------------------------------------------------
# CBraMod model components (from Challenge 2)
# ------------------------------------------------------------
class SincConv1d(nn.Module):
    def __init__(self, out_channels=64, kernel_size=129, sample_rate=100, min_hz=0.3, max_hz=45.0):
        super().__init__()
        self.out_channels = out_channels
        self.kernel_size  = kernel_size
        self.sample_rate  = sample_rate
        self.min_hz       = float(min_hz)
        self.max_hz       = float(max_hz)
        low  = torch.linspace(self.min_hz, self.max_hz - 5.0, out_channels)
        band = torch.ones(out_channels) * 5.0
        self.low_hz_  = nn.Parameter(low)
        self.band_hz_ = nn.Parameter(band)
        n = torch.arange(-(kernel_size // 2), kernel_size // 2 + 1).float()
        self.register_buffer("n", n)

    def forward(self, x):  # x: (B, C, T)
        B, C, T = x.shape
        device, dtype = x.device, x.dtype
        low  = torch.clamp(torch.abs(self.low_hz_), min=self.min_hz, max=self.max_hz - 1.0)
        raw_high = low + torch.abs(self.band_hz_)
        min_v, max_v = low + 1.0, torch.full_like(low, self.max_hz)
        high = torch.clamp(raw_high, min=min_v, max=max_v)
        n = self.n.to(device=device, dtype=dtype)
        window = torch.hamming_window(self.kernel_size, periodic=False, dtype=dtype, device=device)
        nyq = self.sample_rate / 2.0
        filters = []
        for i in range(self.out_channels):
            f1, f2 = low[i]/nyq, high[i]/nyq
            h1 = 2 * f2 * torch.sinc(2 * f2 * n)
            h2 = 2 * f1 * torch.sinc(2 * f1 * n)
            bandpass = (h1 - h2) * window
            filters.append(bandpass)
        filt = torch.stack(filters, dim=0).unsqueeze(1)  # (out, 1, K)
        x_dw = x.view(B * C, 1, T)
        y = F.conv1d(x_dw, filt, stride=1, padding=self.kernel_size // 2)
        y = y.view(B, C, self.out_channels, y.shape[-1]).sum(dim=1)
        return y

class SEBlock(nn.Module):
    def __init__(self, c, r=8):
        super().__init__()
        self.fc1, self.fc2 = nn.Linear(c, c // r), nn.Linear(c // r, c)
    def forward(self, x):
        s = x.mean(-1)
        e = torch.sigmoid(self.fc2(F.relu(self.fc1(s)))).unsqueeze(-1)
        return x * e

class CBraModBackbone(nn.Module):
    def __init__(self, out_dim=512):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(64,128,7,padding=3), nn.ReLU(),
            nn.Conv1d(128,256,5,padding=2), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(256,out_dim)
        self.out_dim = out_dim
    def forward(self,x):
        return self.fc(self.conv(x).squeeze(-1))

# ------------------------------------------------------------
# Regression head (same as in Challenge 1)
# ------------------------------------------------------------
class RtHead(nn.Module):
    def __init__(self, feat_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(feat_dim, max(64, feat_dim//2)),
            nn.ReLU(),
            nn.Linear(max(64, feat_dim//2), 1)
        )
    def forward(self, z):
        if z.ndim > 2: z = torch.flatten(z, 1)
        return self.mlp(z)

# ------------------------------------------------------------
# Load pretrained CBraMod encoder
# ------------------------------------------------------------
def load_cbramod_encoder(weight_path):
    front = SincConv1d(64,129,100)
    se = SEBlock(64)
    backbone = CBraModBackbone(512)
    ckpt = torch.load(weight_path, map_location="cpu")

    front.load_state_dict(ckpt["front"], strict=False)
    backbone.load_state_dict(ckpt["backbone"], strict=False)
    print(f"[INFO] Loaded pretrained CBraMod encoder from {weight_path}")
    
    encoder = nn.Sequential(front, se, backbone)
    return encoder

# ------------------------------------------------------------
# CCD fine-tuning loop (unchanged)
# ------------------------------------------------------------
def train_ccd_rt(dl_tr, encoder, rt_head, epochs=10, lr=1e-3):
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    opt = torch.optim.Adam(list(encoder.parameters()) + list(rt_head.parameters()), lr=lr)
    for ep in range(epochs):
        encoder.train(); rt_head.train(); losses=[]
        for x, y in tqdm(dl_tr, desc=f"[CBraMod→CCD] Epoch {ep+1}/{epochs}"):
            x, y = x.float().to(DEVICE), y.to(DEVICE)
            yhat = rt_head(encoder(x))
            loss = F.l1_loss(yhat, y)
            opt.zero_grad(); loss.backward(); opt.step()
            losses.append(loss.item())
        print(f"Epoch {ep+1}: MAE={np.mean(losses):.3f}")
    return encoder, rt_head

# ------------------------------------------------------------
# Main
# ------------------------------------------------------------
def main():
    # --- Load pretrained CBraMod encoder ---
    encoder = load_cbramod_encoder("/home/RA/EEG_Challenge/Challenge2/best_cbramod_cached_finetune.pth").to("cuda")

    # --- CCD dataset (same as original Challenge 1) ---
    ccd_eeg_files = collect_preprocessed_files(PREPROCESSED_ROOT, task_name="contrastChangeDetection")
    matched_pairs = match_eeg_to_event(ccd_eeg_files, BIDS_ROOT)
    ds_ccd = CcdRtDataset(matched_pairs, win_s=WIN_S_CCD)
    dl_ccd = DataLoader(ds_ccd, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

    # --- Head initialization ---
    rt_head = RtHead(512).to("cuda")

    # --- Fine-tune on CCD ---
    encoder, rt_head = train_ccd_rt(dl_ccd, encoder, rt_head, epochs=EPOCHS_CCD, lr=LR_CCD)

    # --- Save final weights ---
    torch.save({
        "encoder": encoder.state_dict(),
        "rt_head": rt_head.state_dict()
    }, "cbramod_to_ccd_weights.pth")

    print("✅ Saved fine-tuned weights to cbramod_to_ccd_weights.pth")

if __name__ == "__main__":
    main()


[INFO] Loaded pretrained CBraMod encoder from /home/RA/EEG_Challenge/Challenge2/best_cbramod_cached_finetune.pth
[INFO] Found 5386 preprocessed EEG files (task: contrastChangeDetection)
✅ Found 5390 CCD event files.
🔗 Matched 5386 EEG ↔ event pairs.
Building CCD dataset from event files...


Parsing CCD events: 100%|██████████| 5386/5386 [02:08<00:00, 41.95it/s] 


Found 341 total trials. Verifying .npy cache...


Verifying CCD cache: 100%|██████████| 341/341 [00:00<00:00, 3633.16it/s]


✅ CCD Dataset: 341 valid cached trials. RT(ms) Mean=2863.26, Std=168.62


...

Epoch 8: MAE=0.566


[CBraMod→CCD] Epoch 9/10: 100%|██████████| 6/6 [00:04<00:00,  1.29it/s]


Epoch 9: MAE=0.529


[CBraMod→CCD] Epoch 10/10: 100%|██████████| 6/6 [00:06<00:00,  1.05s/it]

Epoch 10: MAE=0.528
✅ Saved fine-tuned weights to cbramod_to_ccd_weights.pth


In [ ]:
# ============================================================
# EEG Foundation Challenge 2025 - Challenge 1
# SuS pretraining → CCD RT regression (100 Hz preprocessed version)
# ------------------------------------------------------------
# - Reads *_eeg_pp.set EEGs (100 Hz) from per-subject folders (run optional)
# - Matches CCD events from BIDS (ds*/sub-*/eeg/*_events.tsv)
# - Caches normalized EEG (.npy)
# - Suppresses MNE/User/Future/Runtime warnings
# - Safe EEGConformer wrapper (version differences)
# ============================================================

import os, random, numpy as np, pandas as pd, warnings, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from glob import glob
import torch.nn.functional as F

# ---- suppress warnings/logs ----
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import mne
mne.set_log_level("ERROR")

# ============================================================
# 0. Config (경로만 맞춰주세요)
# ============================================================
BIDS_ROOT         = "/data5/open_data/HBN/EEG_BIDS/"
PREPROCESSED_ROOT = "/data5/open_data/HBN/Preprocessed_EEG/0922try_bySubject/"
CACHE_DIR         = "/data5/open_data/HBN/cache_eeg_100hz_noref"
os.makedirs(CACHE_DIR, exist_ok=True)

TARGET_SFREQ = 100
WIN_S_SUS, WIN_S_CCD = 2.0, 2.0            # 윈도 길이(초)
STRIDE_S_SUS = 1.0                         # SuS pretrain용 윈도 stride(초)
BATCH_SIZE, NUM_WORKERS = 64, 2
EPOCHS_SUS, EPOCHS_CCD, LR_SUS, LR_CCD = 5, 10, 1e-3, 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ============================================================
# 1) File collectors (run 유무 포함)
# ============================================================
def collect_preprocessed_files(root_path, task_name=None):
    """
    /.../bySubject/sub-XXXX/** 에서 *_eeg_pp.set 수집 (run 유무 무관)
    task_name이 주어지면 해당 문자열 포함 파일만.
    """
    results = []
    for dirpath, _, filenames in os.walk(root_path):
        # subject 폴더만 탐색
        if "sub-" not in dirpath: 
            continue
        for fn in filenames:
            low = fn.lower()
            if not low.endswith("_eeg_pp.set"):
                continue
            if task_name and (task_name.lower() not in low):
                continue
            results.append(os.path.join(dirpath, fn))
    results = sorted(results)
    print(f"[INFO] Found {len(results)} preprocessed EEG files ({'task: '+task_name if task_name else 'all'})")
    return [(f, "") for f in results]

def collect_ccd_event_files(bids_root):
    """BIDS 폴더에서 CCD 이벤트 파일 몽땅 수집."""
    ev_files = glob(os.path.join(
        bids_root, "ds*/sub-*", "eeg", "sub-*_task-contrastChangeDetection*_events.tsv"
    ))
    print(f"✅ Found {len(ev_files)} CCD event files.")
    return ev_files

def match_eeg_to_event(preproc_files, bids_root):
    """
    preprocessed CCD EEG ↔ BIDS 이벤트 매칭.
    규칙: *_eeg_pp.set → *_events.tsv (run 유무 모두 대응)
    """
    ev_files = collect_ccd_event_files(bids_root)
    ev_dict = {os.path.basename(ef).replace("_events.tsv", ""): ef for ef in ev_files}

    pairs = []
    for eeg_path, _ in preproc_files:
        base = os.path.basename(eeg_path)
        # pp 접미어 제거하여 키 생성 (확장자 제거)
        key = base.replace("_eeg_pp.set", "").replace(".set", "")
        if key in ev_dict:
            pairs.append((eeg_path, ev_dict[key]))
        else:
            # run 없는 변형도 탐색
            key_no_run = key.replace("_run-1", "").replace("_run-2", "")
            matched = None
            for k, v in ev_dict.items():
                k_norm = k.replace("_run-1", "").replace("_run-2", "")
                if k_norm == key_no_run:
                    matched = v; break
            if matched:
                pairs.append((eeg_path, matched))
    print(f"🔗 Matched {len(pairs)} EEG ↔ event pairs.")
    return pairs

# ============================================================
# 2) Cached EEG loader (z-score only, no align/resample)
# ============================================================
def read_raw(eeg_path):
    return mne.io.read_raw_eeglab(eeg_path, preload=True, verbose=False)

def cached_load_eeg(eeg_path):
    """파일 단위 캐시: <basename>_cached.npy"""
    fname = os.path.basename(eeg_path).replace(".set", "_cached.npy")
    cache_path = os.path.join(CACHE_DIR, fname)
    if os.path.exists(cache_path):
        return np.load(cache_path)
    raw = read_raw(eeg_path)
    raw.load_data()
    raw.pick_types(eeg=True, meg=False, eog=False, ecg=False, stim=False)
    X = raw.get_data(picks="eeg").astype(np.float32)          # (C,T)
    mean, std = X.mean(1, keepdims=True), X.std(1, keepdims=True) + 1e-6
    X = np.nan_to_num((X - mean) / std)
    np.save(cache_path, X)
    return X

def make_window(x_ct, center_s, sfreq=TARGET_SFREQ, win_sec=2.0):
    """center 이전 win_sec 구간을 잘라 [C,Tw] 반환(부족하면 좌측 패딩)."""
    t1 = int(center_s * sfreq)
    Tw = int(win_sec * sfreq)
    t0 = max(0, t1 - Tw)
    seg = x_ct[:, t0:t1]
    need = Tw - seg.shape[1]
    if need > 0:
        seg = np.pad(seg, ((0, 0), (need, 0)), mode="constant")
    return seg.astype(np.float32)

# ============================================================
# 3) CCD trial parser (correct only: feedback==smiley)
# ============================================================
def extract_ccd_trials(df):
    if df.empty or "onset" not in df.columns or "value" not in df.columns:
        return []
    trials = []
    on  = df["onset"].astype(float).values
    val = df["value"].astype(str).values
    fb  = df["feedback"].astype(str).values if "feedback" in df.columns else ["n/a"] * len(df)
    starts  = [i for i,v in enumerate(val) if "contrastTrial_start" in v]
    presses = [i for i,v in enumerate(val) if "buttonPress" in v]
    for ti in starts:
        t0 = on[ti]
        later = [pi for pi in presses if on[pi] > t0]
        if not later: 
            continue
        pi = later[0]
        rt = (on[pi]-t0) * 1000.0
        if 100 <= rt <= 3000 and "smiley" in fb[pi].lower():
            trials.append((t0, rt))
    return trials

# ============================================================
# 4) Datasets
# ============================================================
class SusPretrainDataset(Dataset):
    """
    이벤트 없이 SuS 파일에서 윈도우를 규칙적/랜덤 추출해 두 view로 반환.
    """
    def __init__(self, eeg_files, win_s=WIN_S_SUS, stride_s=STRIDE_S_SUS, random_start=True):
        self.items = []  # (eeg_path, center_s)
        self.win_s = win_s; self.stride_s = stride_s; self.random_start = random_start
        for p,_ in eeg_files:
            X = cached_load_eeg(p)
            T = X.shape[1]
            Tw = int(win_s * TARGET_SFREQ)
            stride = int(stride_s * TARGET_SFREQ)
            centers = []
            if T > Tw:
                # 센터를 stride 간격으로 전 파일에 깔기
                for t1 in range(Tw, T, stride):
                    centers.append(t1 / TARGET_SFREQ)
            # 파일 내 최소 보장 샘플 수
            if len(centers) == 0 and T >= Tw:
                centers = [Tw / TARGET_SFREQ]
            for c in centers:
                self.items.append((p, c))
        random.shuffle(self.items)

    def __len__(self): return len(self.items)

    @staticmethod
    def _augment(x):
        # 간단한 증강: 가우시안 노이즈 + 타임마스크 + 채널 드롭
        x = x + 0.01 * np.random.randn(*x.shape).astype(np.float32)
        if np.random.rand() < 0.5:
            L = max(1, int(x.shape[1]*0.1))
            s = np.random.randint(0, x.shape[1]-L+1)
            x[:, s:s+L] = 0.0
        if np.random.rand() < 0.5:
            drop = max(1, int(x.shape[0]*0.05))
            idx = np.random.choice(x.shape[0], drop, replace=False)
            x[idx] = 0.0
        return x

    def __getitem__(self, idx):
        p, c = self.items[idx]
        X = cached_load_eeg(p)
        seg = make_window(X, c, win_sec=self.win_s)
        v1 = self._augment(seg.copy())
        v2 = self._augment(seg.copy())
        return torch.from_numpy(v1), torch.from_numpy(v2), torch.zeros(1)

class CcdRtDataset(Dataset):
    def __init__(self, eeg_event_pairs, win_s=WIN_S_CCD):
        self.samples = []  # (eeg_path, onset_s, rt_ms)
        for eeg_path, ev_path in eeg_event_pairs:
            if not os.path.exists(ev_path): 
                continue
            df = pd.read_csv(ev_path, sep="\t")
            for o, rt in extract_ccd_trials(df):
                self.samples.append((eeg_path, o, rt))
        
        # --- (수정) RT 값의 평균과 표준편차 계산 ---
        all_rts = np.array([s[2] for s in self.samples]).astype(np.float32)
        self.rt_mean = all_rts.mean()
        self.rt_std = all_rts.std() + 1e-6 # 0으로 나눠지는 것 방지
        print(f"✅ CCD Dataset: {len(self.samples)} trials. RT(ms) Mean={self.rt_mean:.2f}, Std={self.rt_std:.2f}")
        # ------------------------------------------
        
        self.win_s = win_s

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        p, o, rt = self.samples[idx]
        X = cached_load_eeg(p)
        seg = make_window(X, o, win_sec=self.win_s)
        
        # --- (수정) RT를 Z-score로 정규화 ---
        rt_normalized = (rt - self.rt_mean) / self.rt_std
        # -----------------------------------
        
        return torch.from_numpy(seg), torch.tensor([rt_normalized], dtype=torch.float32) # 정규화된 값 반환

# ============================================================
# 5) EEGConformer encoder (safe wrapper)
# ============================================================
from braindecode.models import EEGConformer

class SafeEEGConformerEncoder(nn.Module):
    """
    EEGConformer 버전별 생성자 차이 자동 대응 + 출력 flatten.
    """
    def __init__(self, n_chans, sfreq, input_window_samples):
        super().__init__()
        last_err = None
        trials = [
            dict(n_chans=n_chans, n_outputs=1, n_times=input_window_samples,              sfreq=sfreq, return_features=True),
            dict(n_chans=n_chans, n_outputs=1, input_window_samples=input_window_samples, sfreq=sfreq, return_features=True),
            dict(n_chans=n_chans, n_outputs=1, n_times=input_window_samples,              sfreq=sfreq),
        ]
        for kw in trials:
            try:
                self.backbone = EEGConformer(**kw)
                break
            except TypeError as e:
                last_err = e
        if not hasattr(self, "backbone"):
            raise TypeError(f"EEGConformer init failed. Last error: {last_err}")

    def forward(self, x):
        z = self.backbone(x)
        if isinstance(z, tuple): z = z[0]
        return torch.flatten(z, 1)

class ContrastiveHead(nn.Module):
    def __init__(self, in_dim, proj_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, in_dim//2),
            nn.ReLU(),
            nn.Linear(in_dim//2, proj_dim)
        )
    def forward(self, x):
        if x.ndim > 2: x = torch.flatten(x, 1)
        return self.net(x)


# ============================================================
# 6) Heads & Losses
# ============================================================
class RtHead(nn.Module):
    def __init__(self, feat_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(feat_dim, max(64, feat_dim//2)),
            nn.ReLU(),
            nn.Linear(max(64, feat_dim//2), 1)
        )
    def forward(self, z):
        if z.ndim > 2: z = torch.flatten(z, 1)
        return self.mlp(z)

def nt_xent_loss(z1, z2, temperature: float = 0.5):
    z1 = F.normalize(z1, dim=1)
    z2 = F.normalize(z2, dim=1)

    logits12 = torch.matmul(z1, z2.T) / temperature   # (N, N)
    logits21 = torch.matmul(z2, z1.T) / temperature   # (N, N)

    # 안정화
    logits12 = logits12 - logits12.max(dim=1, keepdim=True).values
    logits21 = logits21 - logits21.max(dim=1, keepdim=True).values

    labels = torch.arange(z1.size(0), device=z1.device)  # diag가 positive
    loss = (F.cross_entropy(logits12, labels) + F.cross_entropy(logits21, labels)) / 2
    return loss


# ============================================================
# 7) Train loops
# ============================================================
def train_pretrain_sus(dl, encoder, epochs=EPOCHS_SUS, lr=LR_SUS):
    # feature projection 추가
    with torch.no_grad():
        dummy, _, _ = next(iter(dl))
        feat_dim = encoder(dummy[:1].float().to(DEVICE)).shape[1]
    proj_head = nn.Sequential(
        nn.Linear(feat_dim, feat_dim // 2),
        nn.ReLU(),
        nn.Linear(feat_dim // 2, 128)
    ).to(DEVICE)

    opt = torch.optim.Adam(list(encoder.parameters()) + list(proj_head.parameters()), lr=lr)
    for ep in range(epochs):
        encoder.train(); proj_head.train(); losses=[]
        for x1, x2, _ in tqdm(dl, desc=f"[SuS pretrain] {ep+1}/{epochs}"):
            x1, x2 = x1.float().to(DEVICE), x2.float().to(DEVICE)
            z1, z2 = encoder(x1), encoder(x2)
            p1, p2 = proj_head(z1), proj_head(z2)
            loss = nt_xent_loss(p1, p2, temperature=2.0)
            opt.zero_grad(); loss.backward(); opt.step()
            losses.append(loss.item())
        print(f"Epoch {ep+1}: contrastive loss={np.mean(losses):.4f}")
    return encoder


def train_ccd_rt(dl_tr, encoder, rt_head, epochs=EPOCHS_CCD, lr=LR_CCD):
    opt = torch.optim.Adam(list(encoder.parameters()) + list(rt_head.parameters()), lr=lr)
    for ep in range(epochs):
        encoder.train(); rt_head.train(); losses=[]
        for x, y in tqdm(dl_tr, desc=f"[CCD train] {ep+1}/{epochs}"):
            x, y = x.float().to(DEVICE), y.to(DEVICE)
            yhat = rt_head(encoder(x))
            loss = nn.functional.l1_loss(yhat, y)
            opt.zero_grad(); loss.backward(); opt.step()
            losses.append(loss.item())
        print(f"Epoch {ep+1}: MAE={np.mean(losses):.3f} ms")

# ============================================================
# 8) Main
# ============================================================
def main():
    # --- 1) SuS pretraining ---
    all_preproc_files = collect_preprocessed_files(PREPROCESSED_ROOT)

    # task 이름별로 필터링
    sus_like_files = [
        (p, "") for (p, _) in all_preproc_files
        if "contrastchangedetection" not in p.lower()
    ]

    print(f"[INFO] Pretrain files (excluding CCD): {len(sus_like_files)}")

    ds_sus = SusPretrainDataset(
        sus_like_files,
        win_s=WIN_S_SUS,
        stride_s=STRIDE_S_SUS
    )
    dl_sus = DataLoader(
        ds_sus,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True
    )
    # encoder 초기화 (SuS 배치로 C,T 추정)
    x_demo, _, _ = next(iter(dl_sus))
    _, C, T = x_demo.shape
    encoder = SafeEEGConformerEncoder(C, TARGET_SFREQ, T).to(DEVICE)

    # pretrain
    encoder = train_pretrain_sus(dl_sus, encoder, epochs=EPOCHS_SUS, lr=LR_SUS)

    # --- 2) CCD fine-tuning (RT) ---
    ccd_eeg_files = collect_preprocessed_files(PREPROCESSED_ROOT, task_name="contrastChangeDetection")
    matched_pairs = match_eeg_to_event(ccd_eeg_files, BIDS_ROOT)
    ds_ccd = CcdRtDataset(matched_pairs, win_s=WIN_S_CCD)
    dl_ccd = DataLoader(ds_ccd, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

    # head 만들고 학습
    with torch.no_grad():
        feat_dim = encoder(x_demo[:1].float().to(DEVICE)).shape[1]
    print(f"[INFO] Encoder feature dim: {feat_dim}")
    rt_head = RtHead(feat_dim).to(DEVICE)

    train_ccd_rt(dl_ccd, encoder, rt_head, epochs=EPOCHS_CCD, lr=LR_CCD)

    # 학습 종료 후 encoder + rt_head 저장
    torch.save({
        "encoder": encoder.state_dict(),
        "rt_head": rt_head.state_dict()
    }, "weights_ch1.pth")

    print("✅ Saved Challenge 1 weights to weights_ch1.pth")


if __name__ == "__main__":
    main()
